## References:
* https://flask-sqlalchemy.readthedocs.io/en/stable/
* https://docs.sqlalchemy.org/en/20/orm/inheritance.html#concrete-table-inheritance
* https://docs.sqlalchemy.org/en/20/_modules/examples/performance/bulk_inserts.html
* https://docs.sqlalchemy.org/en/20/orm/large_collections.html#bulk-insert-of-new-items
* https://docs.sqlalchemy.org/en/20/orm/extensions/asyncio.html

## Refactoring notes

- The relationship between a frequency severity model and the input premium file is missing
- The back-relationship were missing for handling properly session.delete(histolossfile) :

-

    class Analysis(CommonMixin, Base):
        """Represents an analysis entity."""
    
        id: Mapped[int] = mapped_column(primary_key=True)
        client_id: Mapped[int] = mapped_column(ForeignKey("client.id"), nullable=False)
        client: Mapped["Client"] = relationship(back_populates="analyses")
    
        histolossfiles: Mapped[List["HistoLossFile"]] = relationship(
            secondary=lambda: analysis_histolossfile_table, back_populates="analyses"
        )
        modelfiles: Mapped[List["ModelFile"]] = relationship(
            secondary=lambda: analysis_modelfile_table, back_populates="analyses"
        )

        class HistoLossFile(CommonMixin, Base):
 
        analyses: Mapped[List[Analysis]] = relationship(
            secondary=lambda: analysis_histolossfile_table, back_populates="histolossfiles"
        )

  - 

      class ModelFile(CommonMixin, Base):
            """Base class for model files."""
        
            id: Mapped[int] = mapped_column(primary_key=True)
            model_type: Mapped[str] = mapped_column(String(50), nullable=False)
            years_simulated: Mapped[int] = mapped_column(nullable=False)
        
            client_id: Mapped[int] = mapped_column(ForeignKey("client.id"), nullable=False)
            client: Mapped["Client"] = relationship(back_populates="modelfiles")
        
            yearlosses: Mapped[List["ModelYearLoss"]] = relationship(
                back_populates="modelfile",
                cascade="all, delete-orphan",
            )
        
            analyses: Mapped[List[Analysis]] = relationship(
                secondary=lambda: analysis_modelfile_table, back_populates="modelfiles"
            )
    

- Cascade delete, all for client-analysis:


    class Client(CommonMixin, Base):
        """Represents a client entity."""
    
        id: Mapped[int] = mapped_column(primary_key=True)
        name: Mapped[str] = mapped_column(String(50), nullable=False)
    
        analyses: Mapped[List["Analysis"]] = relationship(
            back_populates="client", cascade="all, delete-orphan"
        )

- The Pydantic classes FrequencyInput and SeverityInput need to be reviewed and refactored
- The attribute treshold is missing in the FrequencyModel class
- Start frequency and severity models parameters with index 0
- The threshold of frequency and severity models is that of the related frequency_severity_model => Remove attribute threshold from severity model
- Use ModelType in polymorphic identity

## Engine

### Enumerations and Dataclasses

In [23]:
from dataclasses import dataclass
from enum import Enum


class ModelType(Enum):
    """Defines the supported loss models."""

    EMPIRICAL = "empirical"
    # FREQUENCY_SEVERITY = "frequency_severity"
    FREQUENCY_SEVERITY = "frequencyseveritymodel"
    COMPOSITE_FREQUENCY_SEVERITY = "composite_frequency_severity"
    EXPOSURE_BASED = "exposure_based"


class DistributionType(Enum):
    """Defines the supported statistical distributions."""

    POISSON = "poisson"
    NEGATIVE_BINOMIAL = "negative_binomial"
    PARETO = "pareto"


class LossType(Enum):
    """Defines the loss types."""

    CAT = "cat"
    NON_CAT = "non_cat"


@dataclass
class DistributionInput:
    """
    Configuration for a statistical distribution.

    Attributes:
        dist: The distribution type (enum).
        params: Parameters specific to the distribution.
    """

    dist: DistributionType
    params: list[float]

### Functions

In [26]:
import numpy as np
import polars as pl
from scipy.stats import poisson, nbinom, pareto


# Main function to generate model year loss
def get_modelyearloss_frequency_severity(
    threshold: float,
    frequency_input: DistributionInput,
    severity_input: DistributionInput,
    simulated_years: int,
    modelfile_id: int,
) -> dict:
    """
    Generate loss data for a frequency-severity model over a number of simulated years.

    Args:
        threshold (float): Multiplier applied to the severity losses.
        frequency_input (DistributionInput): Distribution defining the frequency of events per year.
        severity_input (DistributionInput): Distribution defining the severity of each event.
        simulated_years (int): Number of years to simulate.

    Returns:
        dict: A dictionary with the following keys:
            - "year": List of years for each loss event.
            - "day": Random day of the year for each loss.
            - "loss": Calculated loss values.
            - "loss_type": Type of loss (e.g., catastrophic or non-catastrophic).
    """
    frequencies = generate_frequencies(frequency_input, simulated_years)
    years = generate_years(frequencies)
    loss_count = len(years)
    days = generate_days(loss_count)
    losses = generate_losses_from_parametric_dist(threshold, severity_input, loss_count)
    loss_types = generate_loss_types(loss_count)
    modelfile_ids = np.repeat(modelfile_id, loss_count)

    modelyearloss = pl.DataFrame(
        {
            "year": years,
            "day": days,
            "loss": losses,
            "loss_type": loss_types,
            "modelfile_id": modelfile_ids,
        }
    )
    return modelyearloss


def generate_frequencies(
    frequency_input: DistributionInput,
    size: int,
) -> np.ndarray:
    """
    Generate a list of event frequencies based on a specified distribution.

    Args:
        frequency_input (DistributionInput): The distribution and parameters for frequency generation.
        size (int): Number of values to generate.

    Returns:
        list[int]: A list of event frequencies for each simulated year.
    """
    frequencies = get_sample_from_dist(frequency_input, size)
    return frequencies


def generate_years(frequencies: list[int]) -> list[int]:
    """
    Generate a list of years for loss events based on event frequencies.

    Args:
        frequencies (list[int]): A list where each element represents the number of events in a year.

    Returns:
        list[int]: A list of years, repeated according to their respective frequencies.
    """
    years = [
        year for year, freq in enumerate(frequencies, start=1) for _ in range(freq)
    ]
    return years


def generate_days(size: int) -> np.ndarray:
    """
    Generate random days of the year for loss events.

    Args:
        size (int): Number of days to generate.

    Returns:
        np.ndarray: An array of random integers representing days (1 to 365).
    """
    days = np.random.randint(1, 366, size)
    return days


def generate_loss_types(size: int) -> np.ndarray:
    """
    Generate random loss types (catastrophic or non-catastrophic) for events.

    Args:
        size (int): Number of loss types to generate.

    Returns:
        np.ndarray: An array of randomly chosen loss types (catastrophic or non-catastrophic).
    """
    loss_types = np.random.choice([LossType.CAT.value, LossType.NON_CAT.value], size)
    return loss_types


def generate_losses_from_parametric_dist(
    threshold: int,
    severity_input: DistributionInput,
    loss_count: int,
) -> np.ndarray:
    """
    Generates a set of rounded loss values from a parametric distribution.

    Args:
        threshold (int): A threshold value (not used in the current implementation).
        severity_input (DistributionInput): Parameters defining the severity distribution.
        loss_count (int): The number of loss values to generate.

    Returns:
        np.ndarray: An array of rounded loss values.
    """
    sample = threshold * get_sample_from_dist(severity_input, loss_count)
    return np.round(sample)


def get_sample_from_dist(
    distribution_input: DistributionInput, size: int
) -> np.ndarray:
    """
    Generate a sample from the specified distribution.

    Args:
        distribution_input (DistributionInput): An object containing the distribution type
            (e.g., Poisson, Negative Binomial, Pareto) and its associated parameters.
        size (int): The number of samples to generate.

    Returns:
        np.ndarray: An array of samples drawn from the specified distribution.

    Raises:
        ValueError: If the distribution type is not supported.
    """
    dist = distribution_input.dist
    params = distribution_input.params

    match dist:
        case DistributionType.POISSON:
            return poisson.rvs(mu=params[0], size=size)
        case DistributionType.NEGATIVE_BINOMIAL:
            return nbinom.rvs(n=params[0], p=params[1], size=size)
        case DistributionType.PARETO:
            return pareto.rvs(b=params[0], size=size)
        case _:
            raise ValueError(f"Unsupported distribution: {dist}")


# Example usage
distribution_input = DistributionInput(dist=DistributionType.PARETO, params=[2])
sample = 1000 * get_sample_from_dist(distribution_input, size=1_000_000)

# Output for debugging
print(f"Chosen distribution: {dist.value}")
print(f"Parameters: {params}")

Chosen distribution: pareto
Parameters: [2]


### Tests

In [27]:
import time

threshold = 1000

frequency_input = DistributionInput(
    dist=DistributionType.POISSON,
    params=[4],
)

severity_input = DistributionInput(
    dist=DistributionType.PARETO,
    params=[2],
)

simulated_years = 100_000
modelfile_id = 1
start = time.perf_counter()
modelyearloss = get_modelyearloss_frequency_severity(
    threshold,
    frequency_input,
    severity_input,
    simulated_years,
    modelfile_id,
)
print(f"Duration = {time.perf_counter() - start}")
print(f"Average Loss = {modelyearloss['loss'].mean()}")
print(f"Frequency = {len(modelyearloss['loss']) / simulated_years}")

Duration = 0.5332407999994757
Average Loss = 1996.4408653942141
Frequency = 4.00696


In [28]:
print(modelyearloss)

shape: (400_696, 5)
┌────────┬─────┬────────┬───────────┬──────────────┐
│ year   ┆ day ┆ loss   ┆ loss_type ┆ modelfile_id │
│ ---    ┆ --- ┆ ---    ┆ ---       ┆ ---          │
│ i64    ┆ i32 ┆ f64    ┆ str       ┆ i64          │
╞════════╪═════╪════════╪═══════════╪══════════════╡
│ 1      ┆ 167 ┆ 1165.0 ┆ non_cat   ┆ 1            │
│ 1      ┆ 327 ┆ 1691.0 ┆ cat       ┆ 1            │
│ 1      ┆ 359 ┆ 1152.0 ┆ non_cat   ┆ 1            │
│ 1      ┆ 169 ┆ 1099.0 ┆ cat       ┆ 1            │
│ 1      ┆ 66  ┆ 1041.0 ┆ non_cat   ┆ 1            │
│ …      ┆ …   ┆ …      ┆ …         ┆ …            │
│ 100000 ┆ 212 ┆ 1354.0 ┆ non_cat   ┆ 1            │
│ 100000 ┆ 362 ┆ 1136.0 ┆ cat       ┆ 1            │
│ 100000 ┆ 44  ┆ 1136.0 ┆ cat       ┆ 1            │
│ 100000 ┆ 92  ┆ 9173.0 ┆ cat       ┆ 1            │
│ 100000 ┆ 48  ┆ 1083.0 ┆ cat       ┆ 1            │
└────────┴─────┴────────┴───────────┴──────────────┘


## Backend

### Imports

In [30]:
import time

from sqlalchemy import desc, insert, select, text
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy.orm import Session

from database import (
    Analysis,
    Client,
    HistoLossFile,
    ModelFile,
    ModelYearLoss,
    FrequencyModel,
    SeverityModel,
    FrequencySeverityModel,
    session,
)

### Functions

In [31]:
# Create a client
def create_client(session, client_name):
    """
    Adds a new client to the database.

    Args:
        session (Session): The SQLAlchemy session to use.
        client_name (str): The name of the client to be added.

    Raises:
        SQLAlchemyError: If a database error occurs.
        Exception: If any other unexpected error occurs.
    """
    try:
        # Create a new client
        client = Client(name=client_name)

        # Add the client to the session
        session.add(client)

        # Commit the transaction
        session.commit()
        print(f"Client '{client_name}' added successfully.")

    except SQLAlchemyError as e:
        session.rollback()
        print(f"Database error occurred: {e}")
        raise

    except Exception as e:
        session.rollback()
        print(f"An unexpected error occurred: {e}")
        raise

    finally:
        session.close()


create_client(session, client_name="AXA")

Client 'AXA' added successfully.


In [32]:
# Create an analysis and associate it with a client
def add_analysis_to_client(session, client_id):
    """
    Creates an analysis and associates it with a client.

    Args:
        session (Session): The SQLAlchemy session to use.
        client_id (int): The ID of the client to associate the analysis with.

    Raises:
        SQLAlchemyError: If a database error occurs.
        Exception: If any other unexpected error occurs.
    """
    try:
        # Retrieve the client
        client = session.get_one(Client, client_id)

        # Create a new analysis
        analysis = Analysis()

        # Associate the analysis with the client
        client.analyses.append(analysis)

        # Commit the transaction
        session.commit()
        print("Analysis added successfully")

    except SQLAlchemyError as e:
        session.rollback()
        print(f"Database error occurred: {e}")
        raise

    except Exception as e:
        session.rollback()
        print(f"An unexpected error occurred: {e}")
        raise

    finally:
        session.close()


add_analysis_to_client(session, client_id=1)

Analysis added successfully


In [33]:
# Create a historical loss file and associate it with a client and an analysis
def create_historical_loss_file(session, analysis_id):
    """
    Creates a historical loss file and associates it with a client and an analysis.

    Args:
        session (Session): The SQLAlchemy session to use.
        analysis_id (int): The ID of the analysis to associate the historical loss file with.

    Raises:
        SQLAlchemyError: If a database error occurs.
        Exception: If any other unexpected error occurs.
    """
    try:
        # Retrieve the analysis and the associated client
        analysis = session.get(Analysis, analysis_id)
        if not analysis:
            raise ValueError(f"Analysis with ID {analysis_id} not found.")
        client = analysis.client

        # Create the historical loss file
        histolossfile = HistoLossFile()

        # Associate the historical loss file with the client and analysis
        client.histolossfiles.append(histolossfile)
        analysis.histolossfiles.append(histolossfile)

        # Commit the transaction
        session.commit()
        print("Historical loss file added successfully")

    except SQLAlchemyError as e:
        session.rollback()
        print(f"Database error occurred: {e}")
        raise

    except Exception as e:
        session.rollback()
        print(f"An unexpected error occurred: {e}")
        raise

    finally:
        session.close()


# Call the function
create_historical_loss_file(session, analysis_id=1)

Historical loss file added successfully


In [36]:
# Create a frequency-severity loss model
def create_frequency_severity_model(
    session: Session,
    analysis_id: int,
    lossfile_id: int,
    threshold: float,
    frequency_input: DistributionInput,
    severity_input: DistributionInput,
    years_simulated: int,
) -> None:
    """
    Creates a frequency-severity model and persists related data in the database.

    Args:
        session (Session): The SQLAlchemy session to use for database operations.
        analysis_id (int): ID of the analysis to associate the model with.
        lossfile_id (int): ID of the loss file to associate the model with.
        threshold (float): Threshold parameter for the model.
        frequency_input (DistributionInput): Input parameters for the frequency model.
        severity_input (DistributionInput): Input parameters for the severity model.
        years_simulated (int): Number of years simulated for the model.

    Raises:
        SQLAlchemyError: If a database error occurs during the process.
        Exception: If an unexpected error occurs.
    """
    try:
        # Fetch analysis and ensure it exists
        analysis = session.get(Analysis, analysis_id)
        if not analysis:
            raise ValueError(f"Analysis with ID {analysis_id} not found.")
        client_id = analysis.client_id

        # Create frequency and severity models
        start_time = time.perf_counter()
        frequencymodel = FrequencyModel(
            **{
                f"parameter_{i}": param
                for i, param in enumerate(frequency_input.params)
            }
        )
        severitymodel = SeverityModel(
            **{f"parameter_{i}": param for i, param in enumerate(severity_input.params)}
        )

        # Create the frequency-severity model
        modelfile = FrequencySeverityModel(
            model_type="frequencyseveritymodel",
            threshold=threshold,
            years_simulated=years_simulated,
            lossfile_id=lossfile_id,
            frequencymodel=frequencymodel,
            severitymodel=severitymodel,
        )

        # Link the model to the analysis and the client
        analysis.client.modelfiles.append(modelfile)
        analysis.modelfiles.append(modelfile)
        print(
            f"Time to create the model file in the session: {time.perf_counter() - start_time:.2f} seconds"
        )

        # Flush to get modelfile ID
        start_time = time.perf_counter()
        session.flush()
        print(
            f"Time to flush the session: {time.perf_counter() - start_time:.2f} seconds"
        )

        # Generate year loss data
        start_time = time.perf_counter()
        modelyearloss = get_modelyearloss_frequency_severity(
            threshold, frequency_input, severity_input, years_simulated, modelfile.id
        )
        print(modelyearloss)
        print(
            f"Time to generate model year loss: {time.perf_counter() - start_time:.2f} seconds"
        )

        # Insert records into the database
        start_time = time.perf_counter()
        session.execute(insert(ModelYearLoss), modelyearloss.to_dicts())
        print(
            f"Time to insert model year loss records: {time.perf_counter() - start_time:.2f} seconds"
        )

        # Commit the transaction
        start_time = time.perf_counter()
        session.commit()
        print(
            f"Time to commit transaction: {time.perf_counter() - start_time:.2f} seconds"
        )

    except SQLAlchemyError as e:
        session.rollback()
        print(f"Database error occurred: {e}")
        raise
    except Exception as e:
        session.rollback()
        print(f"An unexpected error occurred: {e}")
        raise
    finally:
        session.close()

In [37]:
# Usage example
start = time.perf_counter()
create_frequency_severity_model(
    session,
    analysis_id=1,
    lossfile_id=1,
    threshold=1000,
    frequency_input=DistributionInput(
        dist=DistributionType.POISSON,
        params=[2, 0, 0, 0, 0],
    ),
    severity_input=DistributionInput(
        dist=DistributionType.PARETO,
        params=[2, 0, 0, 0, 0],
    ),
    years_simulated=100_000,
)

duration = time.perf_counter() - start
print(f"Total Duration = {duration}")

Time to create the model file in the session: 0.01 seconds
Time to flush the session: 0.00 seconds
shape: (200_045, 5)
┌────────┬─────┬────────┬───────────┬──────────────┐
│ year   ┆ day ┆ loss   ┆ loss_type ┆ modelfile_id │
│ ---    ┆ --- ┆ ---    ┆ ---       ┆ ---          │
│ i64    ┆ i32 ┆ f64    ┆ str       ┆ i64          │
╞════════╪═════╪════════╪═══════════╪══════════════╡
│ 1      ┆ 256 ┆ 1150.0 ┆ cat       ┆ 58           │
│ 2      ┆ 275 ┆ 1715.0 ┆ non_cat   ┆ 58           │
│ 3      ┆ 364 ┆ 2509.0 ┆ non_cat   ┆ 58           │
│ 4      ┆ 259 ┆ 2471.0 ┆ cat       ┆ 58           │
│ 4      ┆ 20  ┆ 1020.0 ┆ cat       ┆ 58           │
│ …      ┆ …   ┆ …      ┆ …         ┆ …            │
│ 99994  ┆ 270 ┆ 1097.0 ┆ cat       ┆ 58           │
│ 99996  ┆ 165 ┆ 1031.0 ┆ cat       ┆ 58           │
│ 99996  ┆ 4   ┆ 2792.0 ┆ non_cat   ┆ 58           │
│ 99999  ┆ 98  ┆ 1285.0 ┆ non_cat   ┆ 58           │
│ 100000 ┆ 323 ┆ 1165.0 ┆ cat       ┆ 58           │
└────────┴─────┴────────┴────────

In [21]:
# Delete this cell
query = select(ModelFile).order_by(desc(ModelFile.id)).limit(1)
last_record = session.execute(query).scalars().first()

In [23]:
start = time.perf_counter()
last_record.yearlosses
duration = time.perf_counter() - start
print(f"Duration: {duration}")

Duration: 0.00010379999912402127


In [33]:
# Delete a historical loss file

# Input data
histlossfile_id = 3

try:
    histolossfile = session.get(HistoLossFile, histlossfile_id)
    if not histolossfile:
        raise ValueError(f"Historical loss file with ID {histlossfile_id} not found.")
    session.delete(histolossfile)
    session.commit()
    print(f"The historical loss file has been deleted.")

except SQLAlchemyError as e:
    session.rollback()
    print(f"Database error occurred: {e}")
    raise

except Exception as e:
    session.rollback()
    print(f"An unexpected error occured: {e}")
    raise

finally:
    session.close()

The historical loss file has been deleted.


In [25]:
fsm = session.scalars(select(FrequencySeverityModel)).first()

In [26]:
fsm

FrequencySeverityModel(id=1)

In [34]:
try:
    session.delete(fsm)
    session.commit()
except Exception as e:
    session.rollback()
    print(f"An error occured: {e}")
    raise
finally:
    session.close()

# DELETE A FREQUENCY SEVERITY MODEL, A FREQUENCY SOLO, A SEVERITY SOLO

# TRANSFORM SCRIPTS INTO FUNCTIONS

# THEN CORRECT DAYS RECORDING IN THE DATABASE

# CORRECT IMPLEMENTATION OF THRESHOLD FOR GENERATION LOSSES

# THEN CREATE JIRA SPECIFIC ISSUES

# USE ASYNCIO??? (ASK CHATGPT)

# THEN WORKSHOP WITH ANTOINE B TO REVIEW CHANGES LIKE THOSE IN PYDANTIC FOR FREQUENCYINPUT SEVERITYINPUT ETC

# THEN DO UI FOR TRAIN_SQLA FOR  ENGINE ONLY FOR ACTUARIES